In [7]:
import json
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

with open("results_cluster/05_cluster_label_sentences.json", "r", encoding="utf-8") as f:
    summaries = json.load(f)

labels_list = list(summaries.values())

emb_labels = model.encode(labels_list, normalize_embeddings=True, show_progress_bar=True)
np.save("results_cluster/topic_emb_labels.npy", emb_labels)


Batches: 100%|██████████| 2/2 [00:00<00:00,  2.83it/s]


In [8]:
emb_labels.shape

(51, 768)

In [9]:
from sklearn.cluster import KMeans

n_topics = 7
kmeans = KMeans(n_clusters=n_topics, random_state=42)
topic_ids = kmeans.fit_predict(emb_labels)

df_topics = pd.DataFrame({
    "sentence": labels_list,
    "topic": topic_ids
})
df_topics.to_csv("results_cluster/topic_cluster_labels.csv", index=False)


In [10]:
topic_keywords = {}

for tid in range(n_topics):
    idx = df_topics[df_topics["topic"] == tid].index
    centroid = emb_labels[idx].mean(axis=0)

    sims = emb_labels @ centroid
    top_sentences = [labels_list[i] for i in sims.argsort()[-5:]]

    topic_keywords[tid] = top_sentences

with open("results_cluster/topic_keywords.json", "w", encoding="utf-8") as f:
    json.dump(topic_keywords, f, ensure_ascii=False, indent=2)


In [11]:
for tid, keywords in topic_keywords.items():
    print(f"Topic {tid}:")
    for kw in keywords:
        print(f" - {kw}")
    print()

Topic 0:
 - 9.000 år gamle kunstværker bliver ødelagt af vandrere Ubetænksomme naturgæster er begyndt at udgøre en trussel for Norges kulturarv. Automatisk oplæsning Der er ikke oplæsning af denne artikel, så den oplæses derfor med maskinstemme. Kontakt os gerne på automatiskoplaesning@pol.dk, hvis du hører ord, hvis udtale kan forbedres. Du kan også hjælpe ved at udfylde spørgeskemaet herunder, hvor vi spørger, hvordan du har oplevet den automatiske oplæsning. Spørgeskema om automatisk oplæsning
 - Uden for citat 30. juni kl. 11, tre år og tre dage efter magtovertagelsen, led Mette Frederiksens strategi skibbrud Billedet af den magtfuldkomne Mette Frederiksen, der ikke tager ansvar og hælder sine embedsmænd under busser, bliver forstærket med landsdommer Michael Kistrups rapport. Og det vil blive brugt massivt af de borgerlige i den kommende valgkamp. Automatisk oplæsning Der er ikke oplæsning af denne artikel, så den oplæses derfor med maskinstemme. Kontakt os gerne på automatiskopla

In [13]:
df_topics

,sentence,topic
0,Aarhus Fremad rykker med det uafgjorte resulta...,4
1,Ruslands angreb på Ukraine Efter alt at dømme ...,4
2,Du kan også logge ind via nedenstående tjenest...,2
3,jv.dk bruger cookies til at give dig en god op...,3
4,Efter at det i mere end halvandet år ikke har ...,1
5,"Metacade prisen stabiliseres, idet spiludvikli...",3
6,Coronavirus Regeringen skrotter den automatisk...,4
7,"Et rumkøretøj fra det amerikanske rumagentur, ...",5
8,En ny NASA-sonde er blevet opsendt fra Cape Ca...,5
9,En gruppe amatørastronomer med en ung dansker ...,5
